# Not included in the end-stage of this exercise, this is just a tangential exploration.  #Sidequest mini-game.

---

# tl;dr:
To computers, letters are just fancy-lookin' numbers.  Words can be used in numerical analysis.

In [1]:
import nltk

import pandas as pd

In [1]:
%matplotlib inline

In [ ]:
# Using a pre-generated word-bank, 

In [2]:
df = pd.read_csv('EffectWordNet.tsv',delimiter='\t',header=None)
df.columns = ['some_number','polarity','word','definition']
df['polarity'] = [1 if p == '+Effect' else -1 if p == '-Effect' else 0 for p in df['polarity']]
df = df[['polarity','word']]

pos = df[0:3298]
neg = df[3299:5726]
nul = df[5727:]

pos_words = [','.join(pos['word'].values)]
pos_df = pd.DataFrame([w.split(',') for w in pos_words]).transpose()
pos_df.rename(columns={0:'word'},inplace=True)
pos_df['polarity'] = 1

neg_words = [','.join(neg['word'].values)]
neg_df = pd.DataFrame([w.split(',') for w in neg_words]).transpose()
neg_df.rename(columns={0:'word'},inplace=True)
neg_df['polarity'] = -1

def get_score(tokens):
    score = 0

    for t in tokens:
        if t.lower() in pos_df['word'].values:
            score += 1

        if t.lower() in neg_df['word'].values:
            score -= 1

    return score

In [3]:
base = 'http://www.springfieldspringfield.co.uk/view_episode_scripts.php?tv-show=game-of-thrones&episode='
seasons = 6
episodes = 10

def all_urls(seasons, episodes):
    all_urls = []
    for s in range(1,seasons+1):
        for e in range(1,episodes+1):
            full_url = base+'s{}e{}'.format(format(s,'02d'),format(e,'02d'))
            #print full_url
            all_urls.append(full_url)
    return all_urls

def tokenize_got_script(txt_file):
    contractions = {'\'m':'am','\'re':'are','\'ve':'have','\'d':'would','\'ll':'will','n\'t':'not'}
    tokens = nltk.word_tokenize(txt_file)
    tokens = [t.replace(t,contractions[t]) if t in contractions else t for t in tokens]
    tokens = [t for t in tokens if t.isalpha() == True]
    return tokens

# Getting token frequency - how often is each word said?

In [4]:
token_df = pd.DataFrame()

for url in all_urls(seasons,episodes):
    
    episode = str(url[-5:])
    filename = 'got_script_' + episode + '.txt'
    print episode,
    with open(filename, 'r') as f:
        text = f.read()
        tokens = tokenize_got_script(text.decode('utf-8'))
        tmp = pd.DataFrame()
        tmp['token'] = tokens
        tmp['count'] = 1
        tmp = tmp.groupby('token').count()
        tmp['episode'] = filename
        tmp.reset_index(inplace=True)
        tmp.sort_values(by='count',ascending=False,inplace=True)
        token_df = token_df.append(tmp)

01e01 01e02 01e03 01e04 01e05 01e06 01e07 01e08 01e09 01e10 02e01 02e02 02e03 02e04 02e05 02e06 02e07 02e08 02e09 02e10 03e01 03e02 03e03 03e04 03e05 03e06 03e07 03e08 03e09 03e10 04e01 04e02 04e03 04e04 04e05 04e06 04e07 04e08 04e09 04e10 05e01 05e02 05e03 05e04 05e05 05e06 05e07 05e08 05e09 05e10 06e01 06e02 06e03 06e04 06e05 06e06 06e07 06e08 06e09 06e10


In [5]:
token_df['epID'] = [x[12] + x[-6:-4] for x in token_df['episode']]

# Frequency of a given word

In [6]:
check_for_word = 'father'

token_df[token_df['token'] == check_for_word].sort_values(by='episode',ascending=True).plot(x='epID',
                                                                                            y='count',
                                                                                            title='How Many Times Was The Word "{}" Used in Game of Thrones?'.format(check_for_word))

In [7]:
check_for_word = 'nothing'

token_df[token_df['token'] == check_for_word].sort_values(by='episode',ascending=True).plot(x='epID',
                                                                                            y='count',
                                                                                            title='How Many Times Was The Word "{}" Used in Game of Thrones?'.format(check_for_word))

---

# Finding Top 10 pos + neg words

In [8]:
s_df = pd.read_csv('EffectWordNet.tsv',delimiter='\t',header=None)
s_df.columns = ['some_number','polarity','word','definition']
s_df['polarity'] = [1 if p == '+Effect' else -1 if p == '-Effect' else 0 for p in s_df['polarity']]
s_df = s_df[['polarity','word']]

pos = df[df['polarity'] == 1]
neg = df[df['polarity'] == -1]
nul = df[df['polarity'] == 0]

pos_words = ','.join(pos['word'].values)
tmp_pos = pd.DataFrame([w.split(',') for w in [pos_words]]).transpose()
tmp_pos.rename(columns={0:'word'},inplace=True)
strip_from_pos = ['have','do','will','be','know','lord','can','come','go','back','get','let','tell','mean','fire','cut',
                  'burn','fool','rule','reek','winter','fall','crow','bet','point','refuse','surrender','slave',
                  'escape','obey','abandon','bottom','break','short','pass','strike','assume','judge','harbor',
                  'pig','hunt','hurry','swing','crane','ice','lock','spike','argue','avoid','wench','gutter','dung',
                  'reject','rough','toast','devise','devour','pervert','like','down','take','man','up','want',
                  'make','master']
pos_df = pd.DataFrame()
pos_df['word'] = [w for w in tmp_pos['word'] if w not in strip_from_pos]
pos_df['polarity'] = 1

add_to_neg = ',imp,eunuch,kingslayer,wildling,beast,whore,fire,winter,hunt,pervert,bastard,cunt,fuck,shit,damn,damned,ugly'
neg_words = ','.join(neg['word'].values)
neg_words = neg_words + add_to_neg
tmp_neg = pd.DataFrame([w.split(',') for w in [neg_words]]).transpose()
tmp_neg.rename(columns={0:'word'},inplace=True)
strip_from_neg = ['have','do','be','am','like','at','me','he','be','a','go','get','of','for','l','lt','be','me','he',
                  'give','show','course','wonder','sun','ser','but','even','keep','free','put','return','found','wish',
                  'out','over','bring','bit','work','move','visit','turn','set','fine','sing','support','touch','sight',
                  'vow','grow','experience','book','fish','rise','continue','cook','center','joy','rescue','invite',
                  'inspire','honeycomb','so','up','send','try','can','come','let','tell','make','better','hand',
                  'place','hold','stay','call','best','live','stand','care','light','talk','face','lot','ride','feel',
                  'save','water','start','full','eat','gate','happen','offer','carry','play','mouth','pray','belong',
                  'silver','smart','cat','dog','share','fly','write''wash','counsel','milk','nose','stew','finish',
                  'seat','swing','advise','calm','deliver','floor','attend','cheese','double','seek','cabin','accompany',
                  'weather','badge','flag','say','off','ask','close','wash','require','jug','abstract','occur','devote',
                  'stop','take','down','want']
#retro-adjustment
neg_df = pd.DataFrame()
neg_df['word'] = [w for w in tmp_neg['word'] if w not in strip_from_neg]
neg_df['polarity'] = -1

# check for any missing words in bank

In [9]:
test = 'ugly'

In [10]:
# check to see if a word is in the pos bank

In [11]:
test in pos_df['word'].values

False

In [12]:
# check to see if a word is in the neg bank

In [13]:
test in neg_df['word'].values

True

# Positive Bank QA

# By Episode

In [14]:
df['token_TEST'] = [x.lower() for x in df['token']]

KeyError: 'token'

In [ ]:
df[['token_TEST','count','epID']][df['token_TEST'].isin(pos_df['word'])].sort_values(by='count',ascending=False).to_csv('test.csv',index=False)

# By Token

In [ ]:
df[['token_TEST','count']][df['token_TEST'].isin(pos_df['word'])].sort_values(by='count',ascending=False).groupby('token_TEST').sum().reset_index().to_csv('test.csv',index=False)

# Negative Bank QA

# By Episode

In [ ]:
df['token_TEST'] = [x.lower() for x in df['token']]

In [ ]:
df[['token_TEST','count','epID']][df['token_TEST'].isin(neg_df['word'])].sort_values(by='count',ascending=False).to_csv('test.csv',index=False)

# By Token

In [ ]:
df[['token_TEST','count']][df['token_TEST'].isin(neg_df['word'])].sort_values(by='count',ascending=False).groupby('token_TEST').sum().reset_index().to_csv('test.csv',index=False)

---

# Word Clouds!

In [16]:
import subprocess

In [ ]:
','.join(df['token_TEST'].values)

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

#from os import path

#d = path.dirname(__file__)

# Read the whole text.
#text = open(path.join(d, 'constitution.txt')).read()
text = ','.join(df['token_TEST'].values)
# Generate a word cloud image
wordcloud = WordCloud().generate(text)

# Display the generated image:
# the matplotlib way:

plt.imshow(wordcloud)
plt.axis("off")

In [ ]:
# take relative word frequencies into account, lower max_font_size
wordcloud = WordCloud(max_font_size=400, relative_scaling=.5, width=1000).generate(text)
#plt.figure()
#plt.imshow(wordcloud)
#plt.axis("off")
#plt.show()

# The pil way (if you don't have matplotlib)
image = wordcloud.to_image()
image.show()